# Ollama + OpenAI + Python

## 1. Geef de modelnaam op

Als je een ander model hebt geïmporteerd dan "phi3:mini", wijzig dan de waarde in de cel hieronder. Die variabele zal in de code door het hele notebook worden gebruikt.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Stel de Open AI-client in

Meestal wordt de OpenAI-client gebruikt met OpenAI.com of Azure OpenAI om te communiceren met grote taalmodellen.  
Echter, het kan ook worden gebruikt met Ollama, aangezien Ollama een OpenAI-compatibele endpoint biedt op "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Genereer een chatantwoord

Nu kunnen we de OpenAI SDK gebruiken om een reactie voor een gesprek te genereren. Dit verzoek zou een haiku over katten moeten opleveren:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Prompt engineering

Het eerste bericht dat naar het taalmodel wordt gestuurd, wordt de "systeemboodschap" of "systeemprompt" genoemd, en het stelt de algemene instructies voor het model vast.  
Je kunt je eigen aangepaste systeemprompt geven om een taalmodel te begeleiden bij het genereren van output op een andere manier.  
Pas de `SYSTEM_MESSAGE` hieronder aan om te antwoorden zoals je favoriete beroemde film-/TV-personage, of haal inspiratie voor andere systeemprompts uit [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Zodra je de systeemboodschap hebt aangepast, geef je de eerste gebruikersvraag in de `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Voorbeelden met weinig voorbeelden

Een andere manier om een taalmodel te sturen is door "weinig voorbeelden" te geven, een reeks voorbeeldvragen en -antwoorden die laten zien hoe het moet reageren.

Het onderstaande voorbeeld probeert een taalmodel te laten functioneren als een onderwijsassistent door enkele voorbeelden te geven van vragen en antwoorden die een onderwijsassistent zou kunnen geven, en vervolgens het model te voorzien van een vraag die een student zou kunnen stellen.

Probeer het eerst, en pas daarna de `SYSTEM_MESSAGE`, `EXAMPLES` en `USER_MESSAGE` aan voor een nieuwe situatie.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Retrieval Augmented Generation

RAG (Retrieval Augmented Generation) is een techniek om een taalmodel nauwkeurige antwoorden te laten geven binnen een specifiek domein, door eerst relevante informatie op te halen uit een kennisbron en vervolgens een antwoord te genereren op basis van die informatie.

We hebben een lokale CSV-bestand verstrekt met gegevens over hybride auto's. De onderstaande code leest het CSV-bestand, zoekt naar overeenkomsten met de vraag van de gebruiker en genereert vervolgens een antwoord op basis van de gevonden informatie. Let op dat dit langer zal duren dan de eerdere voorbeelden, omdat er meer gegevens naar het model worden gestuurd. Als je merkt dat het antwoord nog steeds niet gebaseerd is op de gegevens, kun je systeemengineering proberen of andere modellen testen. Over het algemeen is RAG effectiever met grotere modellen of met fijn-afgestemde versies van SLMs.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
